# Vertex AI Search for Retail: Product Catalog Generation

This notebook demonstrates how to use a large language model (LLM) on Vertex AI to generate a synthetic product catalog for a retailer. This is a common first step when building a demo or proof-of-concept for Vertex AI Search for Retail.

**Key Features:**

*   **Modular Configuration:** Uses external files for schema, prompts, and other configurations, making it easy to adapt for different retailers or use cases.
*   **Scalable Data Generation:** Leverages concurrent processing to generate a large number of products efficiently.
*   **Robust JSON Parsing:** Includes logic to handle and clean potential formatting issues in the LLM's JSON output.
*   **BigQuery Integration:** Creates a BigQuery dataset and table, and loads the generated catalog data.

## 1. Setup and Authentication

Install necessary libraries, authenticate to Google Cloud, and set up project-specific variables.

In [4]:
!pip install --upgrade google-cloud-aiplatform google-cloud-bigquery pandas tqdm


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import os
import sys

# If you are running this notebook in a Colab environment, you will need to
# authenticate your user account.
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [6]:
!GOOGLE_CLOUD_PROJECT=`gcloud config set project partarch-ecommerce-demo`


Updated property [core/project].


In [ ]:
# Please set your project details below.
#PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
#GCS_BUCKET_NAME = os.environ.get("GOOGLE_CLOUD_PROJECT")
PROJECT_ID = "partarch-ecommerce-demo"
GCS_BUCKET_NAME = "partarch-ecommerce-demo"
LOCATION = "us-central1"
RETAILER = "wayfair"
MODEL_NAME = "gemini-2.5-pro"

# Generation parameters
NUMBER_OF_PRODUCTS = 1000
PRODUCTS_PER_BATCH = 10 # Number of products to generate in each API call
MAX_WORKERS = 10 # Number of parallel threads for data generation

# BigQuery and GCS configuration
BQ_DATASET = "retail"
BQ_TABLE = f"products-{RETAILER}"
GCS_CATALOG_DIR = f"gs://{GCS_BUCKET_NAME}/retail_catalog/{RETAILER}"
GENERATED_JSONL_GCS_PATH = f"{GCS_CATALOG_DIR}/products.jsonl"

print(f"Project ID: {PROJECT_ID}")
print(f"GCS Bucket: {GCS_BUCKET_NAME}")
print(f"Location: {LOCATION}")
print(f"Retailer: {RETAILER}")
print(f"BigQuery Destination: {PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}")
print(f"GCS Path: {GENERATED_JSONL_GCS_PATH}")

Project ID: None
GCS Bucket: None
Location: us-central1
Retailer: wayfair
BigQuery Destination: None.retail.products-wayfair
GCS Path: gs://None/retail_catalog/wayfair/products.jsonl


In [5]:
import vertexai
from google.cloud import bigquery

vertexai.init(project=PROJECT_ID, location=LOCATION)
bq_client = bigquery.Client(project=PROJECT_ID)

print("Vertex AI and BigQuery clients initialized.")

Vertex AI and BigQuery clients initialized.


## 2. Load Configurations

Load the schema, field requirements, product categories, and the generation prompt from external files. This makes the notebook highly reusable.

In [6]:
# This step creates the necessary configuration files.
# In a real-world scenario, you would upload these files instead of creating them here.

!mkdir -p config
!mkdir -p prompts

# Create schema.json
!echo '[ { "name": "name", "type": "STRING", "mode": "NULLABLE" }, { "name": "id", "type": "STRING", "mode": "REQUIRED" }, { "name": "type", "type": "STRING", "mode": "NULLABLE" }, { "name": "primaryProductId", "type": "STRING", "mode": "NULLABLE" }, { "name": "collectionMemberIds", "type": "STRING", "mode": "REPEATED" }, { "name": "gtin", "type": "STRING", "mode": "NULLABLE" }, { "name": "categories", "type": "STRING", "mode": "REPEATED" }, { "name": "title", "type": "STRING", "mode": "REQUIRED" }, { "name": "brands", "type": "STRING", "mode": "REPEATED" }, { "name": "description", "type": "STRING", "mode": "NULLABLE" }, { "name": "languageCode", "type": "STRING", "mode": "NULLABLE" }, { "name": "attributes", "type": "RECORD", "mode": "REPEATED", "fields": [ { "name": "key", "type": "STRING", "mode": "NULLABLE" }, { "name": "value", "type": "RECORD", "mode": "NULLABLE", "fields": [ { "name": "text", "type": "STRING", "mode": "REPEATED" }, { "name": "numbers", "type": "FLOAT", "mode": "REPEATED" } ] } ] }, { "name": "tags", "type": "STRING", "mode": "REPEATED" }, { "name": "priceInfo", "type": "RECORD", "mode": "NULLABLE", "fields": [ { "name": "currencyCode", "type": "STRING", "mode": "NULLABLE" }, { "name": "price", "type": "FLOAT", "mode": "NULLABLE" }, { "name": "originalPrice", "type": "FLOAT", "mode": "NULLABLE" }, { "name": "cost", "type": "FLOAT", "mode": "NULLABLE" }, { "name": "priceEffectiveTime", "type": "STRING", "mode": "NULLABLE" }, { "name": "priceExpireTime", "type": "STRING", "mode": "NULLABLE" } ] }, { "name": "rating", "type": "RECORD", "mode": "NULLABLE", "fields": [ { "name": "ratingCount", "type": "INTEGER", "mode": "NULLABLE" }, { "name": "averageRating", "type": "FLOAT", "mode": "NULLABLE" }, { "name": "ratingHistogram", "type": "INTEGER", "mode": "REPEATED" } ] }, { "name": "expireTime", "type": "STRING", "mode": "NULLABLE" }, { "name": "ttl", "type": "RECORD", "mode": "NULLABLE", "fields": [ { "name": "seconds", "type": "INTEGER", "mode": "NULLABLE" }, { "name": "nanos", "type": "INTEGER", "mode": "NULLABLE" } ] }, { "name": "availableTime", "type": "STRING", "mode": "NULLABLE" }, { "name": "availability", "type": "STRING", "mode": "NULLABLE" }, { "name": "availableQuantity", "type": "INTEGER", "mode": "NULLABLE" }, { "name": "fulfillmentInfo", "type": "RECORD", "mode": "REPEATED", "fields": [ { "name": "type", "type": "STRING", "mode": "NULLABLE" }, { "name": "placeIds", "type": "STRING", "mode": "REPEATED" } ] }, { "name": "uri", "type": "STRING", "mode": "NULLABLE" }, { "name": "images", "type": "RECORD", "mode": "REPEATED", "fields": [ { "name": "uri", "type": "STRING", "mode": "REQUIRED" }, { "name": "height", "type": "INTEGER", "mode": "NULLABLE" }, { "name": "width", "type": "INTEGER", "mode": "NULLABLE" } ] }, { "name": "audience", "type": "RECORD", "mode": "NULLABLE", "fields": [ { "name": "genders", "type": "STRING", "mode": "REPEATED" }, { "name": "ageGroups", "type": "STRING", "mode": "REPEATED" } ] }, { "name": "colorInfo", "type": "RECORD", "mode": "NULLABLE", "fields": [ { "name": "colorFamilies", "type": "STRING", "mode": "REPEATED" }, { "name": "colors", "type": "STRING", "mode": "REPEATED" } ] }, { "name": "sizes", "type": "STRING", "mode": "REPEATED" }, { "name": "materials", "type": "STRING", "mode": "REPEATED" }, { "name": "patterns", "type": "STRING", "mode": "REPEATED" }, { "name": "conditions", "type": "STRING", "mode": "REPEATED" }, { "name": "publishTime", "type": "STRING", "mode": "NULLABLE" }, { "name": "promotions", "type": "RECORD", "mode": "REPEATED", "fields": [ { "name": "promotionId", "type": "STRING", "mode": "NULLABLE" } ] }]' > config/schema.json

# Create field_requirements.txt
!echo 'name: Immutable. Full resource name of the product, such as projects/*/locations/global/catalogs/default_catalog/branches/default_branch/products/productId.\nid: Immutable. Product identifier, which is the final component of name. For example, this field is "id_1", if name is projects/*/locations/global/catalogs/default_catalog/branches/default_branch/products/id_1. This field must be a UTF-8 encoded string with a length limit of 128 characters.\ntype: Immutable. The type of the product. Must be one of '"'"'PRIMARY'"'"', '"'"'VARIANT'"'"', or '"'"'COLLECTION'"'"'.\nprimaryProductId: Variant group identifier. Must be an id of another product. For PRIMARY products, this field can only be empty or set to the same value as id. For VARIANT products, this field cannot be empty.\ncollectionMemberIds: The id of the collection members when type is '"'"'COLLECTION'"'"'. A maximum of 1000 values are allowed.\ngtin: The Global Trade Item Number (GTIN) of the product. Must be a valid GTIN.\ncategories: Product categories. Use '"'"'>'"'"' to separate hierarchies. Must be set for PRIMARY products. At most 250 values are allowed. Each value must be a UTF-8 encoded string with a length limit of 5,000 characters.\ntitle: Required. Product title. Must be a UTF-8 encoded string with a length limit of 1,000 characters.\nbrands: The brands of the product. A maximum of 30 brands are allowed. Each brand must be a UTF-8 encoded string with a length limit of 1,000 characters.\ndescription: Product description. Must be a UTF-8 encoded string with a length limit of 5,000 characters.\nlanguageCode: Language of the title/description. Use BCP 47 language tags. Defaults to "en-US".\nattributes: Highly encouraged. Extra product attributes. Max 200 entries. Key must match pattern: [a-zA-Z0-9][a-zA-Z0-9_]*. For text attributes, at most 400 values, each with a 256 character limit. For number attributes, at most 400 values.\ntags: Custom tags for filtering. At most 250 values are allowed. Each value must be a UTF-8 encoded string with a length limit of 1,000 characters.\npriceInfo: Product price and cost information.\npriceInfo.currencyCode: The 3-letter currency code defined in ISO 4217.\npriceInfo.price: Price of the product.\npriceInfo.originalPrice: Price of the product without any discount. Should be >= price.\npriceInfo.cost: The costs associated with the sale of a particular product.\npriceInfo.priceEffectiveTime: Timestamp (RFC 3339) when the price starts to be effective.\npriceInfo.priceExpireTime: Timestamp (RFC 3339) when the price stops to be effective.\nrating: The rating of this product.\nrating.ratingCount: The total number of ratings. Must be non-negative.\nrating.averageRating: The average rating of the Product, scaled at 1-5.\nrating.ratingHistogram: List of rating counts per rating value (index = rating - 1). Size must be 5 if non-empty.\nexpireTime: Timestamp (RFC 3339) when the product expires. Must be later than availableTime and publishTime.\nttl: Input only. The TTL (time to live) of the product. e.g. "3600s".\navailableTime: The timestamp (RFC 3339) when this Product becomes available for Search.\navailability: The online availability of the Product. One of '"'"'IN_STOCK'"'"', '"'"'OUT_OF_STOCK'"'"', '"'"'PREORDER'"'"', '"'"'BACKORDER'"'"'. Default to '"'"'IN_STOCK'"'"'.\navailableQuantity: The available quantity of the item.\nfulfillmentInfo: Fulfillment information.\nfulfillmentInfo.type: The fulfillment type. e.g., "pickup-in-store", "ship-to-store", "same-day-delivery", "next-day-delivery", "custom-type-1".\nfulfillmentInfo.placeIds: The IDs for this type, such as store IDs. Max 3000 values.\nuri: Canonical URL directly linking to the product detail page. Length limit of 5,000 characters.\nimages: Product images. Main image first. A maximum of 300 images are allowed.\nimages.uri: Required. URI of the image. Length limit of 5,000 characters.\nimages.height: Height of the image in pixels. Must be non-negative.\nimages.width: Width of the image in pixels. Must be non-negative.\naudience: The target group associated with a given audience.\naudience.genders: Genders of the audience. e.g., "male", "female", "unisex". At most 5 values.\naudience.ageGroups: Age groups of the audience. e.g., "newborn", "infant", "toddler", "kids", "adult". At most 5 values.\ncolorInfo: The color of the product.\ncolorInfo.colorFamilies: Standard color families. e.g., "Red", "Blue", "Green". Max 5 values.\ncolorInfo.colors: The color display names. Max 75 colors.\nsizes: The size of the product. e.g., "S", "M", "L". Max 20 values.\nmaterials: The material of the product. e.g., "leather", "wooden". Max 20 values.\npatterns: The pattern or graphic print of the product. e.g., "striped", "polka dot". Max 20 values.\nconditions: The condition of the product. e.g., "new", "refurbished", "used". Max 1 value.\npublishTime: The timestamp (RFC 3339) when the product is published by the retailer for the first time.\npromotions: The promotions applied to the product. Max 10 values.\npromotions.promotionId: Promotion identifier.' > config/field_requirements.txt

# Create product_categories.txt
!echo 'Furniture > Living Room Furniture > Sofas & Couches\nFurniture > Living Room Furniture > Coffee Tables\nFurniture > Bedroom Furniture > Beds\nFurniture > Bedroom Furniture > Dressers & Chests\nOutdoor > Outdoor Seating > Patio Sofas\nOutdoor > Grills & Outdoor Cooking > Gas Grills\nBed & Bath > Bedding > Comforters & Sets\nBed & Bath > Bath Linens > Bath Towels\nRugs > Area Rugs > Modern Rugs\nDecor & Pillows > Wall Decor > Wall Art\nLighting > Ceiling Fans > Fans with Lights\nKitchen & Tabletop > Cookware > Pots & Pans Sets\nStorage & Organization > Closet Organizers > Closet Systems\nBaby & Kids > Nursery Furniture > Cribs\nHome Improvement > Flooring > Hardwood Flooring' > config/product_categories.txt

# Create data_generation_prompt.txt
!echo 'You are an expert in generating synthetic data for retail product catalogs.\nYour task is to generate a list of {num_products} unique and realistic product entries for the retailer '"'"'{retailer}'"'"'.\n\nThe products should belong to the following categories:\n--- START PRODUCT CATEGORIES ---\n{categories}\n--- END PRODUCT CATEGORIES ---\n\nEach product entry must strictly adhere to the following JSON schema. Do not add any fields that are not in the schema.\n--- START SCHEMA ---\n{schema}\n--- END SCHEMA ---\n\nPay close attention to the following requirements and constraints for each field:\n--- START FIELD REQUIREMENTS ---\n{requirements}\n--- END FIELD REQUIREMENTS ---\n\nIMPORTANT: The entire response must be a single, valid JSON array containing the product objects.\nDo not include any text, explanations, or markdown formatting before or after the JSON array.\nThe output should start with `[` and end with `]`.' > prompts/data_generation_prompt.txt

print("Configuration files created.")

Configuration files created.


In [7]:
import json

def load_text_file(path):
    with open(path, 'r') as f:
        return f.read()

def load_json_file(path):
    with open(path, 'r') as f:
        return json.load(f)

# Load schema and requirements
schema_path = 'config/schema.json'
requirements_path = 'config/field_requirements.txt'
categories_path = 'config/product_categories.txt'
prompt_template_path = 'prompts/data_generation_prompt.txt'

bq_schema = load_json_file(schema_path)
schema_str = json.dumps(bq_schema)
field_requirements = load_text_file(requirements_path)
product_categories = load_text_file(categories_path)
prompt_template = load_text_file(prompt_template_path)

print("Configurations loaded successfully.")
print(f"Loaded {len(bq_schema)} fields in schema.")

Configurations loaded successfully.
Loaded 31 fields in schema.


## 3. Data Generation with Vertex AI Gemini

This section handles the core logic of generating the product catalog. It defines functions to:
1.  Construct a detailed prompt for the LLM.
2.  Call the Gemini API to generate data in batches.
3.  Robustly parse the JSON output, handling potential formatting errors.
4.  Execute the generation process in parallel for scalability and speed.

In [8]:
import re
import time
import concurrent.futures
from tqdm.notebook import tqdm
import vertexai.generative_models as gen_models

def generate_prompt(num_products, retailer, categories, schema, requirements):
    """Constructs the full prompt from the template and inputs."""
    return prompt_template.format(
        num_products=num_products,
        retailer=retailer,
        categories=categories,
        schema=schema,
        requirements=requirements
    )

def clean_and_parse_json(text: str) -> list:
    """
    Cleans the raw text output from the LLM and parses it into a Python list of dicts.
    This function is designed to be robust against common LLM formatting issues,
    such as including markdown code blocks or extra text.
    """
    # Use a regex to find the JSON array within the text
    match = re.search(r'\[.*\]', text, re.DOTALL)
    if not match:
        # Fallback for cases where the model might not use markdown
        start_index = text.find('[')
        end_index = text.rfind(']')
        if start_index != -1 and end_index != -1:
            json_str = text[start_index:end_index+1]
        else:
            raise ValueError("No JSON array found in the model's response.")
    else:
        json_str = match.group(0)

    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Problematic JSON string: {json_str[:500]}...") # Log the problematic part
        raise

def generate_product_batch(batch_num: int, total_batches: int, prompt: str) -> list:
    """
    Generates a single batch of products using the Gemini model.
    Includes retry logic for API errors.
    """
    model = gen_models.GenerativeModel(MODEL_NAME)
    safety_settings = {
        gen_models.HarmCategory.HARM_CATEGORY_HARASSMENT: gen_models.HarmBlockThreshold.BLOCK_NONE,
        gen_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: gen_models.HarmBlockThreshold.BLOCK_NONE,
        gen_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: gen_models.HarmBlockThreshold.BLOCK_NONE,
        gen_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: gen_models.HarmBlockThreshold.BLOCK_NONE,
    }
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1.0,
        "top_p": 0.95,
    }

    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = model.generate_content(
                prompt,
                generation_config=generation_config,
                safety_settings=safety_settings
            )
            return clean_and_parse_json(response.text)
        except (ValueError, json.JSONDecodeError) as e:
            print(f"Batch {batch_num}/{total_batches}: Data parsing error on attempt {attempt + 1}. {e}")
            if attempt == max_retries - 1:
                print(f"Batch {batch_num}/{total_batches}: Failed after {max_retries} attempts.")
                return [] # Return empty list on failure
        except Exception as e:
            print(f"Batch {batch_num}/{total_batches}: API call failed on attempt {attempt + 1} with error: {e}")
            time.sleep(2 ** attempt) # Exponential backoff
            if attempt == max_retries - 1:
                print(f"Batch {batch_num}/{total_batches}: Failed after {max_retries} attempts.")
                return [] # Return empty list on failure
    return [] # Should not be reached, but for safety

In [ ]:
import math
import concurrent.futures

# Calculate the number of batches needed
number_of_batches = math.ceil(NUMBER_OF_PRODUCTS / PRODUCTS_PER_BATCH)
print(f"Total products to generate: {NUMBER_OF_PRODUCTS}")
print(f"Products per batch: {PRODUCTS_PER_BATCH}")
print(f"Number of batches: {number_of_batches}")
print("-" * 30)

# Construct the prompt once
full_prompt = generate_prompt(
    num_products=PRODUCTS_PER_BATCH,
    retailer=RETAILER,
    categories=product_categories,
    schema=schema_str,
    requirements=field_requirements
)

all_products = []
local_jsonl_path = "products.jsonl"

# Use a ThreadPoolExecutor to make parallel API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Open the local file to write results as they complete
    with open(local_jsonl_path, "w") as f:
        # Create a dictionary of futures
        future_to_batch = {
            executor.submit(generate_product_batch, i + 1, number_of_batches, full_prompt): i
            for i in range(number_of_batches)
        }

        # Use tqdm for a progress bar
        pbar = tqdm(concurrent.futures.as_completed(future_to_batch), total=number_of_batches, desc="Generating Product Batches")

        for future in pbar:
            batch_num = future_to_batch[future]
            try:
                product_batch = future.result()
                if product_batch:
                    all_products.extend(product_batch)
                    # Write each product as a new line in the JSONL file
                    for product in product_batch:
                        f.write(json.dumps(product) + '\n')
                    pbar.set_postfix_str(f"Last batch successful. Total products: {len(all_products)}")
                else:
                    pbar.set_postfix_str(f"Batch {batch_num + 1} failed or returned empty.")

            except Exception as exc:
                pbar.set_postfix_str(f"Batch {batch_num + 1} generated an exception: {exc}")

print("-" * 30)
print(f"Data generation complete. Total products generated: {len(all_products)}")
if len(all_products) < NUMBER_OF_PRODUCTS:
    print(f"Warning: The number of generated products ({len(all_products)}) is less than the requested number ({NUMBER_OF_PRODUCTS}). This may be due to API or parsing errors.")

Total products to generate: 1000
Products per batch: 10
Number of batches: 100
------------------------------


/Users/jake.holmquist/colab-catalog-generation/.venv/lib/python3.13/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/Users/jake.holmquist/colab-catalog-generation/.venv/lib/python3.13/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/Users/jake.holmquist/colab-catalog-generation/.venv/lib/python3.13/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For de

Generating Product Batches:   0%|          | 0/100 [00:00<?, ?it/s]

Error decoding JSON: Expecting ',' delimiter: line 607 column 12 (char 14260)
Problematic JSON string: [
  {
    "name": "projects/wayfair-project/locations/global/catalogs/default_catalog/branches/0/products/SKU-AM-Sofa-84",
    "id": "SKU-AM-Sofa-84",
    "type": "PRIMARY",
    "primaryProductId": "SKU-AM-Sofa-84",
    "collectionMemberIds": [],
    "gtin": "0840123456789",
    "categories": [
      "Furniture > Living Room Furniture > Sofas & Couches"
    ],
    "title": "Andover Mills™ Cale 84'' Upholstered Sofa",
    "brands": [
      "Andover Mills™"
    ],
    "description": "This classic ...
Batch 10/100: Data parsing error on attempt 1. Expecting ',' delimiter: line 607 column 12 (char 14260)
Error decoding JSON: Expecting ',' delimiter: line 719 column 8 (char 17466)
Problematic JSON string: [
  {
    "name": "projects/wayfair-project/locations/global/catalogs/default_catalog/branches/default_branch/products/WF-SOFA-8721A",
    "id": "WF-SOFA-8721A",
    "type": "PRIMARY",
  

## 4. Upload to Google Cloud Storage

The generated JSONL file is uploaded to a GCS bucket to be used as a source for the BigQuery load job.

In [ ]:
from google.cloud import storage

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# The destination path in GCS, removing the 'gs://' prefix
destination_blob_name = GENERATED_JSONL_GCS_PATH.replace(f"gs://{GCS_BUCKET_NAME}/", "")

upload_to_gcs(GCS_BUCKET_NAME, local_jsonl_path, destination_blob_name)

## 5. Load Data into BigQuery

This section creates the dataset and table in BigQuery (if they don't already exist) and then loads the data from the GCS file.

In [ ]:
# 1. Create the BigQuery Dataset if it doesn't exist
dataset_id = f"{PROJECT_ID}.{BQ_DATASET}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = LOCATION
try:
    dataset = bq_client.create_dataset(dataset, timeout=30)
    print(f"Created dataset {dataset_id}")
except google.api_core.exceptions.Conflict:
    print(f"Dataset {dataset_id} already exists.")

# 2. Create the BigQuery Table with the specified schema
table_id = f"{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}"
schema = [bigquery.SchemaField.from_api_repr(field) for field in bq_schema]
table = bigquery.Table(table_id, schema=schema)
try:
    table = bq_client.create_table(table)
    print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")
except google.api_core.exceptions.Conflict:
    print(f"Table {table.project}.{table.dataset_id}.{table.table_id} already exists.")

# 3. Load the data from GCS into the BigQuery table
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    schema=schema,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE, # Overwrite table if it exists
)

load_job = bq_client.load_table_from_uri(
    GENERATED_JSONL_GCS_PATH, table_id, job_config=job_config
)

print(f"Starting job {load_job.job_id} to load data into {table_id}")

load_job.result()  # Waits for the job to complete.

destination_table = bq_client.get_table(table_id)
print(f"Load job finished. Loaded {destination_table.num_rows} rows.")

## 6. (Optional) Cleanup

Run the following cell to delete the resources created in this notebook.

In [ ]:
# Set to True to delete the created resources
delete_resources = False #@param {type:"boolean"}

if delete_resources:
    # Delete BigQuery table
    print(f"Deleting BigQuery table: {table_id}")
    bq_client.delete_table(table_id, not_found_ok=True)
    print("Table deleted.")

    # Delete GCS file
    print(f"Deleting GCS file: {GENERATED_JSONL_GCS_PATH}")
    try:
        storage_client = storage.Client(project=PROJECT_ID)
        bucket = storage_client.bucket(GCS_BUCKET_NAME)
        blob = bucket.blob(destination_blob_name)
        blob.delete()
        print("GCS file deleted.")
    except google.api_core.exceptions.NotFound:
        print("GCS file not found, skipping deletion.")
else:
    print("Cleanup skipped. Set 'delete_resources' to True to delete created resources.")